In [1]:
# prompt: install the python spuco package

!pip install spuco


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7026 sha256=bfd6003b6471ff3933873aa070a9a09f326155bac6524fb0e5a40f704744a096
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils


In [8]:
import torch

device = "cuda:2" if torch.cuda.is_available() else "cpu"

In [82]:
# train model using ERM

from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

valset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.99,
    classes=classes,
    split="val"
)
valset.initialize()

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.99,
    classes=classes,
    split="train"
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.99,
    classes=classes,
    split="test"
)
testset.initialize()

100%|██████████| 10000/10000 [00:02<00:00, 4746.92it/s]


In [83]:

import torchvision.transforms as T

T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [84]:
from spuco.models import model_factory

model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)

In [85]:
from spuco.utils import Trainer
from torch.optim import SGD

print(trainset)

erm = Trainer(
    model=model,
    trainset=trainset,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [87]:
erm.train(4)

Epoch 3: 100%|██████████| 751/751 [00:21<00:00, 34.88batch/s, accuracy=100.0%, loss=0.00651]


In [88]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:20,  1.17it/s]

Group (0, 0) Accuracy: 100.0


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:01<00:19,  1.18it/s]

Group (0, 1) Accuracy: 10.401891252955084


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:02<00:15,  1.39it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:15<00:00,  1.26it/s]

Group (4, 3) Accuracy: 0.0


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:15<00:00,  1.59it/s]

Group (4, 4) Accuracy: 100.0


{(0, 0): 100.0,
 (0, 1): 10.401891252955084,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 13.238770685579196,
 (1, 0): 58.67970660146699,
 (1, 1): 100.0,
 (1, 2): 12.990196078431373,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 17.066666666666666,
 (2, 1): 6.666666666666667,
 (2, 2): 100.0,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (3, 0): 5.778894472361809,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 100.0,
 (3, 4): 0.0,
 (4, 0): 52.6448362720403,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 0.0,
 (4, 4): 100.0}

In [89]:
print(evaluator.worst_group_accuracy)
print(evaluator.average_accuracy)
print(evaluator.evaluate_spurious_attribute_prediction())

((0, 2), 0.0)
99.09413479000004
89.15


In [90]:
from spuco.group_inference import Cluster, ClusterAlg

logits = erm.get_trainset_outputs()
cluster = Cluster(
    Z=logits,
    class_labels=trainset.labels,
    cluster_alg=ClusterAlg.KMEANS,
    num_clusters=4,
    device=device,
    verbose=True
)
group_partition = cluster.infer_groups()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Clustering class-wise: 100%|██████████| 5/5 [00:01<00:00,  4.82it/s]


In [91]:
for key in sorted(group_partition.keys()):
    print(key, len(group_partition[key]))

(0, 0) 2517
(0, 1) 633
(0, 2) 2467
(0, 3) 4516
(1, 0) 2883
(1, 1) 2116
(1, 2) 4581
(1, 3) 92
(2, 0) 2125
(2, 1) 4612
(2, 2) 92
(2, 3) 2182
(3, 0) 9649
(3, 1) 20
(3, 2) 25
(3, 3) 53
(4, 0) 1055
(4, 1) 106
(4, 2) 3268
(4, 3) 5012


In [98]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=trainset,
    group_partition=group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:   5%|▌         | 1/20 [00:01<00:32,  1.68s/it]

Group (0, 0) Accuracy: 100.0


Evaluating group-wise accuracy:  10%|█         | 2/20 [00:02<00:25,  1.40s/it]

Group (0, 1) Accuracy: 85.15007898894154


Evaluating group-wise accuracy:  15%|█▌        | 3/20 [00:04<00:24,  1.47s/it]

...

Evaluating group-wise accuracy:  95%|█████████▌| 19/20 [00:21<00:00,  1.17it/s]

Group (4, 2) Accuracy: 98.8984088127295


Evaluating group-wise accuracy: 100%|██████████| 20/20 [00:22<00:00,  1.15s/it]

Group (4, 3) Accuracy: 99.92019154030328


{(0, 0): 100.0,
 (0, 1): 85.15007898894154,
 (0, 2): 99.9189298743413,
 (0, 3): 99.977856510186,
 (1, 0): 100.0,
 (1, 1): 97.16446124763705,
 (1, 2): 99.73804846103471,
 (1, 3): 77.17391304347827,
 (2, 0): 94.25882352941177,
 (2, 1): 99.39288811795316,
 (2, 2): 44.56521739130435,
 (2, 3): 98.16681943171402,
 (3, 0): 91.6468027774899,
 (3, 1): 100.0,
 (3, 2): 88.0,
 (3, 3): 81.13207547169812,
 (4, 0): 97.34597156398104,
 (4, 1): 62.264150943396224,
 (4, 2): 98.8984088127295,
 (4, 3): 99.92019154030328}

In [93]:
from torch.optim import SGD
from spuco.robust_train import GroupBalanceBatchERM
from spuco.models import model_factory

model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
group_balance_erm = GroupBalanceBatchERM(
    model=model,
    num_epochs=5,
    trainset=trainset,
    group_partition=group_partition,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-3, weight_decay=5e-4, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
group_balance_erm.train()

Epoch 4: 100%|██████████| 751/751 [00:22<00:00, 33.32batch/s, accuracy=100.0%, loss=0.0706]


In [94]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:14,  1.63it/s]

Group (0, 0) Accuracy: 99.76359338061465


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:01<00:13,  1.69it/s]

Group (0, 1) Accuracy: 11.82033096926714


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:01<00:13,  1.67it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:15<00:00,  1.63it/s]

Group (4, 3) Accuracy: 84.34343434343434


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:16<00:00,  1.54it/s]

Group (4, 4) Accuracy: 98.73737373737374


{(0, 0): 99.76359338061465,
 (0, 1): 11.82033096926714,
 (0, 2): 2.3640661938534278,
 (0, 3): 38.297872340425535,
 (0, 4): 28.84160756501182,
 (1, 0): 60.14669926650367,
 (1, 1): 99.51100244498778,
 (1, 2): 75.24509803921569,
 (1, 3): 88.23529411764706,
 (1, 4): 55.3921568627451,
 (2, 0): 56.8,
 (2, 1): 17.333333333333332,
 (2, 2): 97.33333333333333,
 (2, 3): 48.266666666666666,
 (2, 4): 11.229946524064172,
 (3, 0): 73.61809045226131,
 (3, 1): 65.74307304785894,
 (3, 2): 59.949622166246854,
 (3, 3): 90.42821158690177,
 (3, 4): 70.27707808564232,
 (4, 0): 70.52896725440806,
 (4, 1): 16.3727959697733,
 (4, 2): 17.38035264483627,
 (4, 3): 84.34343434343434,
 (4, 4): 98.73737373737374}

In [95]:
print(evaluator.worst_group_accuracy)
print(evaluator.average_accuracy)
print(evaluator.evaluate_spurious_attribute_prediction())

((0, 2), 2.3640661938534278)
96.6693947355213
51.08
